In [ ]:
!python -m pip install -U -q 'anthropic[vertex]'

In [2]:
# Please enter your name/initials (no spaces or special characters allowed), ensure that it is unique
UNIQUE_PREFIX="asia-notebooks" ### PLEASE UPDATE THIS

# Cloud project id.
PROJECT_IDS = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_IDS[0]  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = "asia-southeast1"  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output. Fill it without the 'gs://' prefix.
GCS_BUCKET = f"{PROJECT_ID}-{UNIQUE_PREFIX}"  # @param {type:"string"} 
BUCKET_URI = f"gs://{GCS_BUCKET}"  # @param {type:"string"}

# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = !(gcloud config get-value core/account)  # @param {type:"string"}
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]

! gcloud storage buckets create {BUCKET_URI} --project={PROJECT_ID} --location={REGION}


Creating gs://my-project-0004-346516-asia-notebooks/...
ERROR: (gcloud.storage.buckets.create) HTTPError 409: Your previous request to create the named bucket succeeded and you already own it.


In [1]:
import os

import streamlit as st
import vertexai
from vertexai.preview.language_models import TextGenerationModel

PROJECT_ID = os.environ.get("GCP_PROJECT")  # Your Google Cloud Project ID
LOCATION = os.environ.get("GCP_REGION")  # Your Google Cloud Project Region
vertexai.init(project=PROJECT_ID, location=LOCATION)


# @st.cache_resource
def get_model():
    generation_model = TextGenerationModel.from_pretrained("text-bison@002")
    return generation_model


def get_text_generation(prompt="", **parameters):
    generation_model = get_model()
    response = generation_model.predict(prompt=prompt, **parameters)

    return response.text

In [39]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

# input_prompt = """can you give me details of paracetamol"""

def generate(input_prompt):
    model = GenerativeModel("gemini-ultra")
    responses = model.generate_content(
        input_prompt ,
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.2,
        "top_p": 1,
        "top_k": 32
    },
        safety_settings=[],
        stream=True,
    )
    
    all_response  = []
    
    for response in responses:
        # print(response.text, end="")
        all_response.append(response.text)
    
    # print (all_response)
    
    return(" ".join(all_response))
    

def generate_pro(input_prompt):
    model = GenerativeModel("gemini-pro")
    responses = model.generate_content(
    input_prompt,
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.2,
        "top_p": 1
    },stream=True,)
    
    all_response  = []
    
    for response in responses:
        all_response.append(response.text)
    
    # print (all_response)
    
    return(" ".join(all_response))


In [40]:
from google.cloud import aiplatform
from google.cloud.aiplatform.gapic.schema import predict
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

# The AI Platform services require regional API endpoints.
client_options = {"api_endpoint": "us-central1-autopush-aiplatform.sandbox.googleapis.com"}
# Initialize client that will be used to create and send requests.
# This client only needs to be created once, and can be reused for multiple requests.
client = aiplatform.gapic.PredictionServiceClient(
    client_options=client_options
)

def generate_medlpalm(input_prompt):
    instance_dict = {
        "content": input_prompt
    }
    instance = json_format.ParseDict(instance_dict, Value())
    instances = [instance]
    parameters_dict = {
        "maxOutputTokens": 1024,
        "temperature": 0.2,
        "topP": 0.8,
        "topK": 40
    }
    parameters = json_format.ParseDict(parameters_dict, Value())
    response = client.predict(
        endpoint="projects/cloud-llm-preview1/locations/us-central1/publishers/google/models/medpalm2@latest", instances=instances, parameters=parameters,
    )
    print("response")
    
    predictions = response.predictions
    all_response  = []

    for prediction in predictions:
        # print(" prediction:", dict(prediction)["content"])
        all_response.append(dict(prediction)["content"])
        
    return(" ".join(all_response))
    

In [41]:
from google.cloud import aiplatform
from google.cloud.aiplatform.gapic.schema import predict
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

# The AI Platform services require regional API endpoints.
client_options = {"api_endpoint": "us-central1-autopush-aiplatform.sandbox.googleapis.com"}
# Initialize client that will be used to create and send requests.
# This client only needs to be created once, and can be reused for multiple requests.
client = aiplatform.gapic.PredictionServiceClient(
    client_options=client_options
)

def generate_medllms_v1(input_prompt):

    instance_dict = {
        "content": input_prompt
    }
    instance = json_format.ParseDict(instance_dict, Value())
    instances = [instance]
    parameters_dict = {
        "candidateCount": 1,
        "maxOutputTokens": 1024,
        "temperature": 0.2,
        "topP": 0.8,
        "topK": 40
    }
    parameters = json_format.ParseDict(parameters_dict, Value())
    response = client.predict(
        endpoint="projects/cloud-llm-preview1/locations/us-central1/publishers/google/models/medlm-large", instances=instances, parameters=parameters,
    )
    print("response")
    predictions = response.predictions


    all_response  = []

    for prediction in predictions:
        # print(" prediction:", dict(prediction))
        # print(" two" ,  dict(prediction)["content"] )
        all_response.append(dict(prediction)["content"])
        
    return(" ".join(all_response))

In [42]:
import vertexai
from vertexai.language_models import TextGenerationModel

vertexai.init(project="cloud-llm-preview1", location="us-central1")
parameters = {
    "candidate_count": 1,
    "max_output_tokens": 1024,
    "temperature": 1,
    "top_k": 40
}

def generate_palm_unicorn_v1(input_prompt):
    
    model = TextGenerationModel.from_pretrained("text-unicorn@001")

    response = model.predict(
        input_prompt,
        **parameters
    )
    print(f"Response from Model: {response.text}")
    
    return(response.text)

def generate_palm_bison32k(input_prompt):
    
    model = TextGenerationModel.from_pretrained("text-bison-32k")

    response = model.predict(
        input_prompt,
        **parameters
    )
    print(f"Response from Model: {response.text}")
    
    return(response.text)


### Anthropic model now included

In [14]:
from anthropic import AnthropicVertex

LOCATION="us-central1" # or "europe-west4"

client = AnthropicVertex(region=LOCATION, project_id=PROJECT_ID)

def generate_anthropic_response(input_prompt):
  """Generates a text response using the AnthropicVertex client.

  Args:
    input_prompt: The text prompt for the model.

  Returns:
    The generated text response.
  """
  message = client.messages.create(
    max_tokens=1024,
    messages=[
      {
        "role": "user",
        "content": input_prompt,
      }
    ],
    model="claude-3-haiku@20240307",  # or any other desired model
  )

  response_text = message.model_dump_json(indent=2)  # Accessing response
  print(f"Response from Model: {response_text}")

  return response_text 

def generate_anthropic_response_sonnet(input_prompt):
  """Generates a text response using the AnthropicVertex client.

  Args:
    input_prompt: The text prompt for the model.

  Returns:
    The generated text response.
  """
  message = client.messages.create(
    max_tokens=1024,
    messages=[
      {
        "role": "user",
        "content": input_prompt,
      }
    ],
    # model="claude-3-haiku@20240307",  # or any other desired model
    model = "claude-3-sonnet@20240229" 
  )

  response_text = message.model_dump_json(indent=2)  # Accessing response
  print(f"Response from Model: {response_text}")

  return response_text 

In [15]:
# input_prompt = "What are the symptoms of influenza?" 

# x = generate_pro(input_prompt)
# generate_medllms_v1(input_prompt)
# generate_palm_unicorn_v1(input_prompt)
# input_prompt = "What are the symptoms of influenza?" 
# generate_medlpalm(input_prompt)

In [16]:
!ls

healthcare_gemini_models_v1.ipynb


### First file

In [35]:
import pandas as pd
filename = "./data/Anaes 16 Clinical scenarios RAG response.csv"
df = pd.read_csv(filename)
df['combine_prompt'] = df['System Prompts'] + ' ' +df['RAG Results'] + ' '+ df['User Question']


# print(df['System Prompts'], df['RAG Results'] ,df['User Question'] )
# print(selected_column[0])

In [36]:
# generate_medlpalm(df.loc[0 ,'combine_prompt'])

In [37]:
for i in range(0, len(df)):

    
    if i<=100:
        # df['Gemini_ultra_model_output'][i] = generate(df['combine_prompt'][i])
        print("iteration #", i)

    df.loc[i, "Gemini_ultra_model_output"] = generate_pro(df.loc[i,'combine_prompt'])
    df.loc[i, "Gemini_ultra_model_output_v1"] = generate_pro(df.loc[i,'combine_prompt'])
    df.loc[i, "Gemini_ultra_model_output_v2"] = generate_pro(df.loc[i,'combine_prompt'])
    
    df.loc[i, "Anthropic_haiko_model_output"] = generate_anthropic_response(df.loc[i,'combine_prompt'])

    # df.loc[i, "medLM_model_output"] = generate_medllms_v1(df.loc[i,'combine_prompt'])
    df.loc[i, "medpalm_model_output"] = generate_medlpalm(df.loc[i,'combine_prompt'])
    df.loc[i, "palm_bison32k_output"] = generate_palm_bison32k(df.loc[i,'combine_prompt'])

# generate_medllms_v1(input_prompt)
# generate_palm_unicorn_v1(input_prompt)
# input_prompt = "What are the symptoms of influenza?" 
# generate_medlpalm(input_prompt)    
    
# print( "/n output here ::" , df['Gemini_ultra_model_output'][i])
# df = df.assign(Gemini_ultra_model_output=generate(df.combine_prompt))
# df['combine_prompt'].head(3)

# df['Gemini_ultra_model_output'].head(3)


iteration # 0
['**Medical Colleague Instructions:**\n\n1. Doctor\n2. Fast for 6 hours for solids and 2 hours for clear fluids prior to surgery.\n3. No\n4. **Medication Instructions:**\n   - Continue Ventolin PRN as needed.\n5. NA\n6. **Preoperative', ' Optimization:**\n   - Diabetes: Refer to Internal Medicine for optimization of blood glucose control.\n7. No\n8. **Department Protocols:**\n   - Guidelines on Preoperative Assessment of Obese Patients:\n     - Discuss risks of difficult airway, major adverse cardiac events, and other complications related to obesity.\n', '     - Prescribe gastroprophylaxis with ranitidine.\n   - ERAS Spine Protocol:\n     - Optimize blood glucose control if HbA1c > 8.5%.\n   - Anaesthesia Protocol for ERAS Spine:\n     - Consider IV ketamine bolus at induction.\n     -', ' Administer IV paracetamol at the end of surgery.\n     - Administer IV dexamethasone for PONV prophylaxis if not diabetic or if HbA1c < 8.\n     - Administer IV ondansetron at the end 

In [50]:
# df.columns

In [51]:
# df columnsname(columns={'generate_pro_model_output': 'Gemini_pro_model_output'})  
# df = df.rename(columns={'generate_pro_model_output': 'Gemini_pro_model_output'})  


In [49]:

# Delete the 'col2' column
df = df.drop('combine_prompt', axis=1)

output1 = "./output/O_Anaes 16 Clinical scenarios RAG response_output_gemini.csv"

df.to_csv(output1)


### Second file

In [52]:
import pandas as pd
filename2 = "./data/Anaes 16 Clinical scenarios response base.csv"
df = pd.read_csv(filename2)
df['combine_prompt'] = df['System Prompts'] + ' '+ df['User Prompts']


for i in range(0, len(df)):

    if i<=100:
        # df['Gemini_ultra_model_output'][i] = generate(df['combine_prompt'][i])
        print("iteration #", i)

    # df.loc[i, "Gemini_ultra_model_output"] = generate(df.loc[i,'combine_prompt'])
    # df.loc[i, "Gemini_ultra_model_output_v1"] = generate(df.loc[i,'combine_prompt'])
    # df.loc[i, "Gemini_ultra_model_output_v2"] = generate(df.loc[i,'combine_prompt'])
    df.loc[i, "Gemini_pro_model_output"] = generate_pro(df.loc[i,'combine_prompt'])

    df.loc[i, "Anthropic_haiko_model_output"] = generate_anthropic_response(df.loc[i,'combine_prompt'])

    
    # df.loc[i, "medLM_model_output"] = generate_medllms_v1(df.loc[i,'combine_prompt'])
    df.loc[i, "medpalm_model_output"] = generate_medlpalm(df.loc[i,'combine_prompt'])
    df.loc[i, "palm_bison32k_output"] = generate_palm_bison32k(df.loc[i,'combine_prompt'])
    df.loc[i, "palm_unicorn_output"] = generate_palm_unicorn_v1(df.loc[i,'combine_prompt'])

    # print( "/n output here ::" , df['Gemini_ultra_model_output'][i])

df = df.drop('combine_prompt', axis=1)

output2 = "./output/O_Anaes 16 Clinical scenarios response base response_output_gemini.csv"

df.to_csv(output2)


iteration # 0
response
response
Response from Model:  **Preoperative anesthesia instructions**

**1. Should the patient be seen by a Doctor or Nurse?**
Doctor

**2. Fasting instructions**
- NPO for solids after midnight
- Clear fluids until 6 hours before the surgery

**3. Suitability for preoperative carbohydrate loading**
No

**4. Medication instructions**
- Ventolin: Continue using Ventolin as needed up until the morning of surgery.

**5. Instructions for the healthcare team**
- Arrange for preoperative blood group matching.
- Consider standby postoperative high dependency/ICU bed due to the patient's severe OSA.

**6. Any preoperative optimization required for the patient**
- Diabetes control: The patient's HbA1c is elevated and she has not seen her doctor for over a year. She should be seen by her doctor for diabetes management and optimization prior to surgery.

**7. Any need to delay the operation for further medical workup and preoperative optimization?**
No

**8. Any specific 

### third file Pharmbot1

In [55]:
import pandas as pd
filename3 = "./data/Pharmbot 23 Clinical scenarios RAG response.csv"
df = pd.read_csv(filename3)
df['combine_prompt1'] = df['System Prompts'] + ' ' +df['RAG Results 1'] + ' '+ df['User Question']
df['combine_prompt2'] = df['System Prompts'] + ' ' +df['RAG Results 2'] + ' '+ df['User Question']
df['combine_prompt3'] = df['System Prompts'] + ' ' +df['RAG Results 3'] + ' '+ df['User Question']


for i in range(0, len(df)):

    if i<=100:
        print("iteration #", i)

    # df.loc[i, "Gemini_ultra_model_output1"] = generate(df.loc[i,'combine_prompt1'])
    # df.loc[i, "Gemini_ultra_model_output2"] = generate(df.loc[i,'combine_prompt2'])
    # df.loc[i, "Gemini_ultra_model_output3"] = generate(df.loc[i,'combine_prompt3'])

    df.loc[i, "Gemini_pro_model_output1"] = generate_pro(df.loc[i,'combine_prompt1'])
    df.loc[i, "Gemini_pro_model_output2"] = generate_pro(df.loc[i,'combine_prompt2'])
    df.loc[i, "Gemini_pro_model_output3"] = generate_pro(df.loc[i,'combine_prompt3'])

    # df.loc[i, "medLM_model_output"] = generate_medllms_v1(df.loc[i,'combine_prompt'])
    df.loc[i, "medpalm_model_output1"] = generate_medlpalm(df.loc[i,'combine_prompt1'])
    df.loc[i, "medpalm_model_output2"] = generate_medlpalm(df.loc[i,'combine_prompt2'])
    df.loc[i, "medpalm_model_output3"] = generate_medlpalm(df.loc[i,'combine_prompt3'])
    
    df.loc[i, "palm_bison32k_output1"] = generate_palm_bison32k(df.loc[i,'combine_prompt1'])
    df.loc[i, "palm_bison32k_output2"] = generate_palm_bison32k(df.loc[i,'combine_prompt2'])
    df.loc[i, "palm_bison32k_output3"] = generate_palm_bison32k(df.loc[i,'combine_prompt3'])

    df.loc[i, "Anthropic_haiko_model_output1"] = generate_anthropic_response(df.loc[i,'combine_prompt1'])

    # df.loc[i, "palm_unicorn_output"] = generate_palm_unicorn_v1(df.loc[i,'combine_prompt'])
    
    # print( "/n output here ::" , df['Gemini_ultra_model_output'][i])


iteration # 0
response
response
response
Response from Model:  Situation: The patient's current medication regimen includes multiple medications for various conditions. There are potential drug-related problems, including drug interactions, therapeutic duplication, omission of therapy, and dose adjustments needed due to renal impairment.

Background: The patient has a history of poorly controlled diabetes mellitus, hypertension, hyperlipidemia, chronic kidney disease stage 4, necrotising fasciitis of the right lower limb, left foot osteomyelitis, large right medial thigh abscess, recurrent deep vein thrombosis and pulmonary embolism, and evolved anterior myocardial infarction. His current medications include Enoxaparin Sodium Injection, Actrapid, Lantus, Novorapid, Aspirin, Clopidogrel, Omeprazole, Glyceryl Trinitrate, Linagliptin, Bisoprolol Fumarate, Perindopril Erbumine, and Neurobion.

Assessment:
1. Drug Interactions:
- Enoxaparin Sodium and Aspirin: Both have antiplatelet propert

In [56]:
df = df.drop('combine_prompt1', axis=1)
df = df.drop('combine_prompt2', axis=1)
df = df.drop('combine_prompt3', axis=1)


output3 = "./output/O_Pharmbot 23 Clinical scenarios RAG response response_output_gemini.csv"

df.to_csv(output3)


#### fourth file for Pharmbot2

In [58]:
import pandas as pd
filename4 = "./data/Pharmbot 23 Clinical scenarios response base.csv"
df = pd.read_csv(filename4)
df['combine_prompt'] = df['System Prompts'] + ' ' + df['User Prompts']


for i in range(0, len(df)):

    if i<=100:
        print("iteration #", i)

    # df.loc[i, "Gemini_ultra_model_output"] = generate(df.loc[i,'combine_prompt'])
    # df.loc[i, "Gemini_ultra_model_output_v1"] = generate(df.loc[i,'combine_prompt'])
    # df.loc[i, "Gemini_ultra_model_output_v2"] = generate(df.loc[i,'combine_prompt'])
    
    df.loc[i, "Gemini_pro_model_output"] = generate_pro(df.loc[i,'combine_prompt'])
    df.loc[i, "Anthropic_haiko_model_output"] = generate_anthropic_response(df.loc[i,'combine_prompt'])

    # df.loc[i, "medLM_model_output"] = generate_medllms_v1(df.loc[i,'combine_prompt'])
    df.loc[i, "medpalm_model_output"] = generate_medlpalm(df.loc[i,'combine_prompt'])
    df.loc[i, "palm_bison32k_output"] = generate_palm_bison32k(df.loc[i,'combine_prompt'])
    df.loc[i, "palm_unicorn_output"] = generate_palm_unicorn_v1(df.loc[i,'combine_prompt'])

df = df.drop('combine_prompt', axis=1)

output4 = "./output/O_Pharmbot 23 Clinical scenarios response base response_output_gemini.csv"

df.to_csv(output4)


iteration # 0
response
response
Response from Model:  **Situation:**

The patient has poorly controlled diabetes, hypertension, hyperlipidemia, CKD Stage 2, and a history of recurrent deep vein thrombosis (DVT) and pulmonary embolism (PE). He is currently taking warfarin, but it was held due to his recent coronary angiography and stent placement. Enoxaparin was started as a bridge anticoagulant. The patient is also taking aspirin and clopidogrel for secondary prevention of cardiovascular events.

**Background:**

The patient has a history of poorly controlled diabetes, which can increase the risk of cardiovascular disease and kidney disease. He is also taking multiple medications that may interact with each other, including warfarin, enoxaparin, aspirin, and clopidogrel.

**Assessment:**

- **Drug-drug interaction:** The patient is taking warfarin, which is a vitamin K antagonist that can interact with several other medications. Enoxaparin, aspirin, and clopidogrel can all increase the

### Trying the Claude Anthropic model